# **Mounting Google Drive**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# mounting the google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Installing required library**

In [3]:
# install transformer for tokenizer
!pip install transformers
# install contraction library to replace short word , I'll => I will 
!pip install contractions

     |████████████████████████████████| 2.3MB 28.3MB/s 
     |████████████████████████████████| 901kB 31.8MB/s 
     |████████████████████████████████| 3.3MB 33.6MB/s 
     |████████████████████████████████| 327kB 29.6MB/s 
     |████████████████████████████████| 286kB 35.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85395 sha256=abce0cfc7db46287d666489e29a6acdcbe2cc728e7a8dc138296e0228df8d8e5
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


# **Importing the packages**

In [4]:
# import pickle  and load_model 
from tensorflow.keras.models import load_model
import pickle

# using tokenizer to encode sentence into input_Ids and attention mask
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#Importing packages from Tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [5]:
#importing required libraries
import nltk
import json
import inflect
import contractions
from textblob import TextBlob 
from bs4 import BeautifulSoup
import re, string, unicodedata
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd

In [6]:
# download the necessary nltk packages
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# **Text Preprocessing Functon**

In [7]:
# First function is used to denoise text
def denoise_text(text):
    # Strip html if any. For ex. removing <html>, <p> tags
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    # Replace contractions in the text. For ex. didn't -> did not
    text = contractions.fix(text)
    return text
    
# Check the function 
sample_text = "<p>he didn't say anything </br> about what's gonna <html> happen in the climax"
denoise_text(sample_text)

'he did not say anything  about what is going to  happen in the climax'

In [8]:
def cont_exp(x):
  abbreviations_path="/content/gdrive/MyDrive/FY_Project/abbreviations_wordlist.json"
  abbreviations = json.load(open(abbreviations_path))
  if type(x) is str:
    for key in abbreviations:
      value = abbreviations[key]
      raw_text = r'\b' + key + r'\b'
      x = re.sub(raw_text, value, x)
      # print(raw_text,value, x)
    return x
  else:
    return x

cont_exp("project report complete asap. gn i am out lol")

'project report complete as soon as possible. good night i am out laughing out loud'

In [9]:
# Text normalization includes many steps.

# Gets rid of the non ASCII characters
def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

# Converts the entire text to lowercase
def to_lowercase(words):
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

# Removes the punctuation marks
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

# Numbers to their textual representation
def replace_numbers(words):
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

# Removes stop words
def remove_stopwords(words):
    new_words = []
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    for word in words:
        if word not in all_stopwords:
            new_words.append(word)
    return new_words

# Lemmatizes verbs in the list of tokenized words
def lemmatize_verbs(words):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word)
        lemmas.append(lemma)
    return lemmas

#All above functions are put together
def normalize_text1(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    words = lemmatize_verbs(words)
    return words

#All above functions are put together
def normalize_text2(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    #words = remove_stopwords(words)
    words = lemmatize_verbs(words)
    return words


# **Text Preparation for input data**

In [10]:
# Tokenize tweet into words
def nltk_tokenize(text):
    return nltk.word_tokenize(text)

#Text Preparation to use them into svm and bilstm model
def text_prepare(text):
    text = denoise_text(text)
    text = cont_exp(text)
    text = ' '.join([x for x in normalize_text1(nltk_tokenize(text))])
    return text

#Text Preparation to use them into bert-lstm model
def text_prepare_2(text):
    text = denoise_text(text)
    text = cont_exp(text)
    text = ' '.join([x for x in normalize_text2(nltk_tokenize(text))])
    return text

# **Verifying Preprocessing Function**

In [11]:
# testing preprocessing function
sample_text = 'he didn\'t say anything!!!😊      about that man @harry_kane   what is going to  happen #india. covid-19 patients are increasing expotentially?????? !!'

print(sample_text)
print("\ntokenize output =>", nltk_tokenize(sample_text))
print("\npreprocessed output =>", text_prepare_2(sample_text))

he didn't say anything!!!😊      about that man @harry_kane   what is going to  happen #india. covid-19 patients are increasing expotentially?????? !!

tokenize output => ['he', 'did', "n't", 'say', 'anything', '!', '!', '!', '😊', 'about', 'that', 'man', '@', 'harry_kane', 'what', 'is', 'going', 'to', 'happen', '#', 'india', '.', 'covid-19', 'patients', 'are', 'increasing', 'expotentially', '?', '?', '?', '?', '?', '?', '!', '!']

preprocessed output => he did not say anything about that man harry_kane what is going to happen india covid19 patient are increasing expotentially


# **Google Drive Path for Our Model**

In [12]:
# model path
svm_path='/content/gdrive/MyDrive/FY_Project/Models/SVM/'
lstm_path='/content/gdrive/MyDrive/FY_Project/Models/BiLSTM/'
bert_path='/content/gdrive/MyDrive/FY_Project/Models/BERT_LSTM/'

# **Loading All Model From Drive**

**SVM**

In [13]:
# loading all SVM model
with open(svm_path+'levelA','rb') as f:
  svm1,tf1=pickle.load(f)
with open(svm_path+'levelB','rb') as f:
  svm2,tf2=pickle.load(f)
with open(svm_path+'levelC','rb') as f:
 svm3,tf3=pickle.load(f)

**BiLSTM**

In [14]:
# loading tokenizer1 for bilstm level 1
with open(lstm_path+'tokenizer1', 'rb') as f:
    tokenizer1 = pickle.load(f)

# loading tokenizer1 for bilstm level 2
with open(lstm_path+'tokenizer2', 'rb') as f:
    tokenizer2 = pickle.load(f)

# loading tokenizer1 for bilstm level 3
with open(lstm_path+'tokenizer3', 'rb') as f:
    tokenizer3 = pickle.load(f)

# cut-off score obtained of level1 and leve2 for trained bilstm 
mycutoff1=.49
mycutoff2=.45

# loading all BiLSTM model
BiLSTM_ModelA = load_model(lstm_path+'levelA')
BiLSTM_ModelB = load_model(lstm_path+'levelB')
BiLSTM_ModelC = load_model(lstm_path+'levelC')



**BERT-LSTM**

In [15]:
# encoding input sentence
def bert_tokenize(sent):
    tokens =  tokenizer.encode_plus(
    text=sent,  # the sentence to be encoded
    add_special_tokens=True,  # Add [CLS] and [SEP]
    truncation=True,  # add truncation
    max_length = 50,  # length of a sentence
    padding='max_length',  # Add [PAD]s
    return_token_type_ids=False, # not returning token id
    return_attention_mask = True,  # Generate the attention mask
    return_tensors = 'tf',  # ask the function to return tensorflow tensors
    ) 
    return tokens['input_ids'], tokens['attention_mask']

# loading all BERT model
BERT_ModelA = load_model(bert_path+'levelA')
BERT_ModelB = load_model(bert_path+'levelB')
BERT_ModelC = load_model(bert_path+'levelC')

# **Three Level Class**

In [16]:
# three level classification
ClassA=['NOT','OFF']
ClassB=['UNT','TIN']
ClassC=['OTH','IND','GRP']

# **SVM Model Predict Function**

**Level 1**

In [17]:
# for level 1
def svm_level1(instance):
  instance=text_prepare(instance)
  instance_tf = tf1.transform([instance])
  instance_tf_pd=pd.DataFrame(instance_tf.toarray(),columns=tf1.get_feature_names())
  pred =  svm1.predict(instance_tf_pd)
  score=pred[0]
  label=ClassA[pred[0]]
  return label,score

**Level 2**

In [18]:
# for level 2
def svm_level2(instance):
  instance=text_prepare(instance)
  instance_tf = tf2.transform([instance])
  instance_tf_pd=pd.DataFrame(instance_tf.toarray(),columns=tf2.get_feature_names())
  pred =  svm2.predict(instance_tf_pd)
  score=pred[0]
  label=ClassB[pred[0]]
  return label,score

**Level 3**

In [19]:
# for level 3
def svm_level3(instance):
  instance=text_prepare(instance)
  instance_tf = tf3.transform([instance])
  instance_tf_pd=pd.DataFrame(instance_tf.toarray(),columns=tf3.get_feature_names())
  pred =  svm3.predict(instance_tf_pd)
  score=pred[0]
  label=ClassC[pred[0]]
  return label,score

# **BiLSTM Model Predict Function**

**Level 1**

In [20]:
# for level 1 
def bilstm_level1(instance):
  instance=text_prepare(instance)
  instance_seq = tokenizer1.texts_to_sequences([instance])
  instance_padded = pad_sequences(instance_seq, padding='post', maxlen=100)
  pred=BiLSTM_ModelA.predict(instance_padded)
  score=np.amax(pred)
  label=''
  if(score<=mycutoff1):
    label=ClassA[0]
  else:
    label=ClassA[1]
  return label,score

**Level 2**

In [21]:
# for level 2
def bilstm_level2(instance):
  instance=text_prepare(instance)
  instance_seq = tokenizer2.texts_to_sequences([instance])
  instance_padded = pad_sequences(instance_seq, padding='post', maxlen=100)
  pred=BiLSTM_ModelB.predict(instance_padded)
  score=np.amax(pred)
  label=''
  if(score<=mycutoff2):
    label=ClassB[0]
  else:
    label=ClassB[1]
  return label,score

**Level 3**

In [22]:
# for level 3
def bilstm_level3(instance):
  instance=text_prepare(instance)
  instance_seq = tokenizer3.texts_to_sequences([instance])
  instance_padded = pad_sequences(instance_seq, padding='post', maxlen=100)
  pred=BiLSTM_ModelC(instance_padded)
  score=np.amax(pred)
  label=ClassC[np.argmax(pred,axis=1)[0]]
  return label,score

# **BERT+LSTM Model Predict Function**

**Level 1**

In [23]:
# for level 1
def bert_level1(sent):
    sent=text_prepare_2(sent)
    encoded=bert_tokenize(sent)
    pred=BERT_ModelA(encoded)
    score=np.amax(pred)
    label=ClassA[np.argmax(pred,axis=1)[0]]
    return label,score

**Level 2**

In [24]:
# for level 2
def bert_level2(sent):
    sent=text_prepare_2(sent)
    encoded=bert_tokenize(sent)
    pred=BERT_ModelB(encoded)
    score=np.amax(pred)
    label=ClassB[np.argmax(pred,axis=1)[0]]
    return label,score



**Level 3**

In [25]:
# for level 3
def bert_level3(sent):
    sent=text_prepare_2(sent)
    encoded=bert_tokenize(sent)
    pred=BERT_ModelC(encoded)
    score=np.amax(pred)
    label=ClassC[np.argmax(pred,axis=1)[0]]
    return label,score

# **Classify Function**

In [26]:
def svm_classify(sent):
  print("SVM =>")
  print("-----------------------------------------")
  # level A
  label1,score1=svm_level1(sent)
  print(f"Level A: {label1}, score: {score1}")
  if label1=='OFF':
    # level B
    label2,score2=svm_level2(sent)
    print(f"Level B: {label2}, score: {score2}")
    if label2=='TIN':
      # level C
      label3,score3=svm_level3(sent)
      print(f"Level C: {label3}, score: {score3}")
  print("-----------------------------------------\n")

In [27]:
def bilstm_classify(sent):
  print("BiLSTM  =>")
  print("-----------------------------------------")
  # level A
  label1,score1=bilstm_level1(sent)
  print(f"Level A: {label1}, score: {score1:.2f}")
  if label1=='OFF':
    # level B
    label2,score2=bilstm_level2(sent)
    print(f"Level B: {label2},  score: {score2:.2f}")
    if label2=='TIN':
      # level C
      label3,score3=bilstm_level3(sent)
      print(f"Level C: {label3}, score: {score3:.2f}")
  print("-----------------------------------------\n")

In [28]:
def bert_classify(sent):
  print("BERT-LSTM =>")
  print("-----------------------------------------")
  # level A
  label1,score1=bert_level1(sent)
  print(f"Level A: {label1}, score: {score1:.2f}")
  if label1=='OFF':
    # level B
    label2,score2=bert_level2(sent)
    print(f"Level B: {label2},  score: {score2:.2f}")
    if label2=='TIN':
      # level C
      label3,score3=bert_level3(sent)
      print(f"Level C: {label3}, score: {score3:.2f}")
  print("-----------------------------------------\n")

# **Testing Input data**

In [29]:
tweet=" what the hell , what you think are doing here "

In [30]:
svm_classify(tweet)
bilstm_classify(tweet)
bert_classify(tweet)

SVM =>
-----------------------------------------
Level A: NOT, score: 0
-----------------------------------------

BiLSTM  =>
-----------------------------------------
Level A: OFF, score: 0.66
Level B: TIN,  score: 0.57
Level C: IND, score: 0.83
-----------------------------------------

BERT-LSTM =>
-----------------------------------------
Level A: OFF, score: 0.80
Level B: TIN,  score: 0.55
Level C: IND, score: 0.45
-----------------------------------------

